# Opponent Quality Analysis Dashboard

How do elite players perform against different levels of competition? This analysis explores win rates, 
performance trends, and head-to-head records against opponents categorized by ranking tier.

**Opponent Tiers:**
- **Elite (Top 10)**: The absolute best in the world
- **Strong (11-25)**: Consistent top performers
- **Solid (26-50)**: Quality tour players
- **Mid-tier (51-100)**: Competitive professionals
- **Lower (100+)**: Developing or lower-ranked players

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data
atp_matches = pd.read_csv('../../data/top25/atp/atp_top25_matches.csv')
wta_matches = pd.read_csv('../../data/top25/wta/wta_top25_matches.csv')
atp_players = pd.read_csv('../../data/top25/atp/atp_top25_players.csv')
wta_players = pd.read_csv('../../data/top25/wta/wta_top25_players.csv')

# Create player name lookup
atp_player_names = dict(zip(atp_players['player_id'], 
                            atp_players['name_first'] + ' ' + atp_players['name_last']))
wta_player_names = dict(zip(wta_players['player_id'], 
                            wta_players['name_first'] + ' ' + wta_players['name_last']))

# Add tour identifier
atp_matches['tour'] = 'ATP'
wta_matches['tour'] = 'WTA'

print(f"ATP matches: {len(atp_matches):,}")
print(f"WTA matches: {len(wta_matches):,}")

ATP matches: 8,904
WTA matches: 7,293


In [3]:
def categorize_opponent_tier(rank):
    """Categorize opponent based on their ranking at match time"""
    if pd.isna(rank):
        return 'Unknown'
    rank = int(rank)
    if rank <= 10:
        return 'Elite (1-10)'
    elif rank <= 25:
        return 'Strong (11-25)'
    elif rank <= 50:
        return 'Solid (26-50)'
    elif rank <= 100:
        return 'Mid-tier (51-100)'
    else:
        return 'Lower (100+)'

def process_matches(matches_df, player_ids, player_names):
    """Process matches to create player-centric view with opponent tiers"""
    records = []
    
    for _, row in matches_df.iterrows():
        winner_id = row['winner_id']
        loser_id = row['loser_id']
        
        # If winner is in our top 25
        if winner_id in player_ids:
            records.append({
                'player_id': winner_id,
                'player_name': player_names.get(winner_id, row['winner_name']),
                'opponent_id': loser_id,
                'opponent_name': row['loser_name'],
                'opponent_rank': row['loser_rank'],
                'opponent_tier': categorize_opponent_tier(row['loser_rank']),
                'won': 1,
                'surface': row['surface'],
                'tourney_level': row['tourney_level'],
                'tourney_name': row['tourney_name'],
                'tourney_date': row['tourney_date'],
                'round': row['round'],
                'player_rank': row['winner_rank'],
                'tour': row['tour']
            })
        
        # If loser is in our top 25
        if loser_id in player_ids:
            records.append({
                'player_id': loser_id,
                'player_name': player_names.get(loser_id, row['loser_name']),
                'opponent_id': winner_id,
                'opponent_name': row['winner_name'],
                'opponent_rank': row['winner_rank'],
                'opponent_tier': categorize_opponent_tier(row['winner_rank']),
                'won': 0,
                'surface': row['surface'],
                'tourney_level': row['tourney_level'],
                'tourney_name': row['tourney_name'],
                'tourney_date': row['tourney_date'],
                'round': row['round'],
                'player_rank': row['loser_rank'],
                'tour': row['tour']
            })
    
    return pd.DataFrame(records)

# Process both tours
atp_player_ids = set(atp_players['player_id'])
wta_player_ids = set(wta_players['player_id'])

atp_records = process_matches(atp_matches, atp_player_ids, atp_player_names)
wta_records = process_matches(wta_matches, wta_player_ids, wta_player_names)

# Parse dates
atp_records['date'] = pd.to_datetime(atp_records['tourney_date'], format='%Y%m%d')
wta_records['date'] = pd.to_datetime(wta_records['tourney_date'], format='%Y%m%d')

# Combine
all_records = pd.concat([atp_records, wta_records], ignore_index=True)

print(f"ATP player match records: {len(atp_records):,}")
print(f"WTA player match records: {len(wta_records):,}")

ATP player match records: 10,091
WTA player match records: 8,136


## View 1: Win Rate by Opponent Tier Heatmap

This heatmap shows each player's win percentage against different tiers of opponents. 
Darker colors indicate higher win rates. Numbers in cells show match counts for context.

In [4]:
def create_tier_heatmap(records_df, title, min_matches=5):
    """Create a heatmap of win rates by opponent tier"""
    # Filter out unknown tiers
    df = records_df[records_df['opponent_tier'] != 'Unknown'].copy()
    
    # Calculate win rates by player and tier
    tier_stats = df.groupby(['player_name', 'opponent_tier']).agg(
        wins=('won', 'sum'),
        matches=('won', 'count')
    ).reset_index()
    tier_stats['win_pct'] = (tier_stats['wins'] / tier_stats['matches'] * 100).round(1)
    
    # Get overall win rate for sorting
    overall = df.groupby('player_name').agg(
        total_wins=('won', 'sum'),
        total_matches=('won', 'count')
    ).reset_index()
    overall['overall_win_pct'] = overall['total_wins'] / overall['total_matches']
    
    # Pivot for heatmap
    tier_order = ['Elite (1-10)', 'Strong (11-25)', 'Solid (26-50)', 'Mid-tier (51-100)', 'Lower (100+)']
    
    pivot_pct = tier_stats.pivot(index='player_name', columns='opponent_tier', values='win_pct')
    pivot_matches = tier_stats.pivot(index='player_name', columns='opponent_tier', values='matches')
    
    # Reorder columns
    pivot_pct = pivot_pct[[c for c in tier_order if c in pivot_pct.columns]]
    pivot_matches = pivot_matches[[c for c in tier_order if c in pivot_matches.columns]]
    
    # Sort by overall win rate
    player_order = overall.sort_values('overall_win_pct', ascending=True)['player_name'].tolist()
    pivot_pct = pivot_pct.reindex([p for p in player_order if p in pivot_pct.index])
    pivot_matches = pivot_matches.reindex([p for p in player_order if p in pivot_matches.index])
    
    # Create annotations with match counts
    annotations = []
    for i, player in enumerate(pivot_pct.index):
        for j, tier in enumerate(pivot_pct.columns):
            pct = pivot_pct.loc[player, tier]
            matches = pivot_matches.loc[player, tier]
            if pd.notna(pct) and pd.notna(matches) and matches >= min_matches:
                text = f"{pct:.0f}%<br>({int(matches)})"
            elif pd.notna(matches) and matches > 0:
                text = f"({int(matches)})"
            else:
                text = ""
            annotations.append(
                dict(
                    x=tier, y=player,
                    text=text,
                    showarrow=False,
                    font=dict(size=9, color='white' if (pd.notna(pct) and pct > 60) else 'black')
                )
            )
    
    # Mask cells with too few matches
    masked_pct = pivot_pct.copy()
    masked_pct[pivot_matches < min_matches] = np.nan
    
    fig = go.Figure(data=go.Heatmap(
        z=masked_pct.values,
        x=masked_pct.columns.tolist(),
        y=masked_pct.index.tolist(),
        colorscale='RdYlGn',
        zmin=0,
        zmax=100,
        colorbar=dict(title='Win %', ticksuffix='%'),
        hovertemplate='Player: %{y}<br>Tier: %{x}<br>Win Rate: %{z:.1f}%<extra></extra>'
    ))
    
    fig.update_layout(
        title=dict(text=title, font=dict(size=16)),
        xaxis_title='Opponent Tier',
        yaxis_title='Player',
        height=700,
        annotations=annotations
    )
    
    return fig

# Create heatmaps for both tours
fig_atp = create_tier_heatmap(atp_records, 'ATP: Win Rate by Opponent Tier')
fig_atp.show()

In [5]:
fig_wta = create_tier_heatmap(wta_records, 'WTA: Win Rate by Opponent Tier')
fig_wta.show()

## View 2: Expected vs Actual Performance

This scatter plot compares actual win rates against expected win rates based on ranking differential.
Players above the diagonal are **overperformers** (winning more than expected), 
while those below are **underperformers**.

In [6]:
def expected_win_prob(player_rank, opponent_rank):
    """Calculate expected win probability based on ranking differential
    Using a simplified Elo-style model based on rank difference"""
    if pd.isna(player_rank) or pd.isna(opponent_rank):
        return 0.5
    # Use log-rank difference for better scaling
    rank_diff = np.log(opponent_rank) - np.log(player_rank)
    # Sigmoid function to convert to probability
    return 1 / (1 + np.exp(-rank_diff * 0.8))

def create_expected_vs_actual(records_df, title, min_matches=30):
    """Create scatter plot of expected vs actual performance"""
    df = records_df.copy()
    df = df[df['opponent_rank'].notna() & df['player_rank'].notna()]
    
    # Calculate expected win probability for each match
    df['expected_win'] = df.apply(
        lambda x: expected_win_prob(x['player_rank'], x['opponent_rank']), axis=1
    )
    
    # Aggregate by player
    player_stats = df.groupby('player_name').agg(
        actual_wins=('won', 'sum'),
        expected_wins=('expected_win', 'sum'),
        matches=('won', 'count')
    ).reset_index()
    
    player_stats['actual_pct'] = player_stats['actual_wins'] / player_stats['matches'] * 100
    player_stats['expected_pct'] = player_stats['expected_wins'] / player_stats['matches'] * 100
    player_stats['delta'] = player_stats['actual_pct'] - player_stats['expected_pct']
    
    # Filter by minimum matches
    player_stats = player_stats[player_stats['matches'] >= min_matches]
    
    # Create scatter plot
    fig = px.scatter(
        player_stats,
        x='expected_pct',
        y='actual_pct',
        text='player_name',
        size='matches',
        color='delta',
        color_continuous_scale='RdYlGn',
        color_continuous_midpoint=0,
        hover_data={'matches': True, 'delta': ':.1f'},
        labels={
            'expected_pct': 'Expected Win %',
            'actual_pct': 'Actual Win %',
            'delta': 'Performance Delta',
            'matches': 'Total Matches'
        },
        title=title
    )
    
    # Add diagonal line (y = x)
    fig.add_trace(go.Scatter(
        x=[40, 100], y=[40, 100],
        mode='lines',
        line=dict(dash='dash', color='gray'),
        name='Expected = Actual',
        showlegend=False  # Hide from legend - line is self-explanatory
    ))

    fig.update_traces(
        textposition='top center',
        selector=dict(mode='markers+text')
    )

    fig.update_layout(
        height=600,
        xaxis=dict(range=[40, 100]),
        yaxis=dict(range=[40, 100])
    )
    
    return fig

fig_exp_atp = create_expected_vs_actual(atp_records, 'ATP: Expected vs Actual Win Rate')
fig_exp_atp.show()

In [7]:
fig_exp_wta = create_expected_vs_actual(wta_records, 'WTA: Expected vs Actual Win Rate')
fig_exp_wta.show()

## View 3: Performance Drop-off Lines

This chart shows how each player's win rate changes as opponent quality increases.
**Flat lines** indicate "elite beaters" who maintain performance against top opponents.
**Steep drop-offs** indicate "stat padders" who dominate weaker opponents but struggle vs the best.

In [8]:
def create_dropoff_chart(records_df, title, highlight_player=None, min_matches=5):
    """Create line chart showing win rate across opponent tiers"""
    df = records_df[records_df['opponent_tier'] != 'Unknown'].copy()
    
    tier_order = ['Lower (100+)', 'Mid-tier (51-100)', 'Solid (26-50)', 'Strong (11-25)', 'Elite (1-10)']
    
    # Calculate win rates
    tier_stats = df.groupby(['player_name', 'opponent_tier']).agg(
        wins=('won', 'sum'),
        matches=('won', 'count')
    ).reset_index()
    tier_stats['win_pct'] = tier_stats['wins'] / tier_stats['matches'] * 100
    
    # Filter by min matches per tier
    tier_stats = tier_stats[tier_stats['matches'] >= min_matches]
    
    # Order tiers
    tier_stats['tier_order'] = tier_stats['opponent_tier'].map({t: i for i, t in enumerate(tier_order)})
    tier_stats = tier_stats.sort_values(['player_name', 'tier_order'])
    
    fig = go.Figure()
    
    players = tier_stats['player_name'].unique()
    
    for player in players:
        player_data = tier_stats[tier_stats['player_name'] == player]
        is_highlighted = highlight_player and player == highlight_player
        
        fig.add_trace(go.Scatter(
            x=player_data['opponent_tier'],
            y=player_data['win_pct'],
            mode='lines+markers',
            name=player,
            line=dict(
                width=3 if is_highlighted else 1,
                color='red' if is_highlighted else 'rgba(150,150,150,0.3)'
            ),
            marker=dict(size=8 if is_highlighted else 4),
            hovertemplate=f'{player}<br>Tier: %{{x}}<br>Win Rate: %{{y:.1f}}%<extra></extra>',
            visible=True,
            legendgroup=player
        ))
    
    # Create dropdown for player selection
    buttons = [dict(
        label='All Players',
        method='update',
        args=[{'line.width': [1] * len(players),
               'line.color': ['rgba(150,150,150,0.5)'] * len(players)}]
    )]
    
    for i, player in enumerate(players):
        widths = [1] * len(players)
        colors = ['rgba(150,150,150,0.3)'] * len(players)
        widths[i] = 4
        colors[i] = 'red'
        buttons.append(dict(
            label=player,
            method='update',
            args=[{'line.width': widths, 'line.color': colors}]
        ))
    
    fig.update_layout(
        title=title,
        xaxis_title='Opponent Tier (Weakest → Strongest)',
        yaxis_title='Win Rate (%)',
        height=600,
        showlegend=False,
        updatemenus=[dict(
            active=0,
            buttons=buttons,
            x=1.0,
            xanchor='right',
            y=1.02,
            yanchor='bottom',
            showactive=True,
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='#ccc',
            borderwidth=1,
            font=dict(size=11),
            pad=dict(l=10, t=10)
        )],
        xaxis=dict(categoryorder='array', categoryarray=tier_order),
        margin=dict(t=80)
    )
    
    return fig

fig_dropoff_atp = create_dropoff_chart(atp_records, 'ATP: Performance Drop-off by Opponent Tier')
fig_dropoff_atp.show()

In [9]:
fig_dropoff_wta = create_dropoff_chart(wta_records, 'WTA: Performance Drop-off by Opponent Tier')
fig_dropoff_wta.show()

## View 4: Strength of Schedule

Which players face the toughest competition? This chart shows the average opponent ranking faced by each player.

In [10]:
def create_strength_of_schedule(records_df, title):
    """Create bar chart of average opponent ranking"""
    df = records_df[records_df['opponent_rank'].notna()].copy()
    
    # Calculate stats
    sos_stats = df.groupby('player_name').agg(
        avg_opp_rank=('opponent_rank', 'mean'),
        median_opp_rank=('opponent_rank', 'median'),
        matches=('won', 'count'),
        wins=('won', 'sum')
    ).reset_index()
    sos_stats['win_pct'] = sos_stats['wins'] / sos_stats['matches'] * 100
    
    # Sort by average opponent rank (lower = tougher schedule)
    sos_stats = sos_stats.sort_values('avg_opp_rank')
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=sos_stats['player_name'],
        y=sos_stats['avg_opp_rank'],
        marker_color=sos_stats['win_pct'],
        marker_colorscale='RdYlGn',
        marker_cmin=50,
        marker_cmax=90,
        marker_colorbar=dict(title='Win %'),
        hovertemplate='<b>%{x}</b><br>' +
                      'Avg Opp Rank: %{y:.1f}<br>' +
                      'Win Rate: %{marker.color:.1f}%<br>' +
                      '<extra></extra>',
        text=[f"{r:.0f}" for r in sos_stats['avg_opp_rank']],
        textposition='outside'
    ))
    
    fig.update_layout(
        title=title,
        xaxis_title='Player',
        yaxis_title='Average Opponent Ranking',
        xaxis_tickangle=-45,
        height=500,
        yaxis=dict(autorange='reversed')  # Lower rank = tougher, so reverse
    )
    
    return fig

fig_sos_atp = create_strength_of_schedule(atp_records, 'ATP: Strength of Schedule (Avg Opponent Ranking)')
fig_sos_atp.show()

In [11]:
fig_sos_wta = create_strength_of_schedule(wta_records, 'WTA: Strength of Schedule (Avg Opponent Ranking)')
fig_sos_wta.show()

## View 5: Surface x Opponent Quality Matrix

Where do players struggle most? This matrix shows win rates across surface types AND opponent tiers,
revealing specific weaknesses like "struggles vs top 10 on hard courts".

In [12]:
def create_surface_tier_matrix(records_df, player_name, title):
    """Create surface x opponent tier matrix for a specific player"""
    df = records_df[
        (records_df['player_name'] == player_name) & 
        (records_df['opponent_tier'] != 'Unknown') &
        (records_df['surface'].isin(['Hard', 'Clay', 'Grass']))
    ].copy()
    
    if len(df) == 0:
        return None
    
    tier_order = ['Elite (1-10)', 'Strong (11-25)', 'Solid (26-50)', 'Mid-tier (51-100)', 'Lower (100+)']
    surface_order = ['Hard', 'Clay', 'Grass']
    
    # Calculate stats
    matrix_stats = df.groupby(['surface', 'opponent_tier']).agg(
        wins=('won', 'sum'),
        matches=('won', 'count')
    ).reset_index()
    matrix_stats['win_pct'] = matrix_stats['wins'] / matrix_stats['matches'] * 100
    
    # Pivot
    pivot_pct = matrix_stats.pivot(index='surface', columns='opponent_tier', values='win_pct')
    pivot_matches = matrix_stats.pivot(index='surface', columns='opponent_tier', values='matches')
    
    # Reorder
    pivot_pct = pivot_pct.reindex(index=surface_order, columns=tier_order)
    pivot_matches = pivot_matches.reindex(index=surface_order, columns=tier_order)
    
    # Create annotations
    annotations = []
    for i, surface in enumerate(pivot_pct.index):
        for j, tier in enumerate(pivot_pct.columns):
            pct = pivot_pct.loc[surface, tier]
            matches = pivot_matches.loc[surface, tier]
            if pd.notna(pct) and pd.notna(matches) and matches >= 1:
                text = f"{pct:.0f}%<br>({int(matches)})"
                annotations.append(
                    dict(x=tier, y=surface, text=text, showarrow=False,
                         font=dict(size=11, color='white' if pct > 60 else 'black'))
                )
    
    fig = go.Figure(data=go.Heatmap(
        z=pivot_pct.values,
        x=pivot_pct.columns.tolist(),
        y=pivot_pct.index.tolist(),
        colorscale='RdYlGn',
        zmin=0,
        zmax=100,
        colorbar=dict(title='Win %')
    ))
    
    fig.update_layout(
        title=f"{title}: {player_name}",
        xaxis_title='Opponent Tier',
        yaxis_title='Surface',
        height=350,
        annotations=annotations
    )
    
    return fig

def create_surface_tier_comparison(records_df, tour_name, min_matches=20):
    """Create dropdown-based surface x tier comparison"""
    players = records_df['player_name'].unique()
    
    # Start with first player
    fig = create_surface_tier_matrix(records_df, players[0], f'{tour_name} Surface x Opponent Tier')
    
    # Create dropdown
    buttons = []
    for player in sorted(players):
        df = records_df[
            (records_df['player_name'] == player) & 
            (records_df['opponent_tier'] != 'Unknown') &
            (records_df['surface'].isin(['Hard', 'Clay', 'Grass']))
        ]
        
        if len(df) < min_matches:
            continue
            
        tier_order = ['Elite (1-10)', 'Strong (11-25)', 'Solid (26-50)', 'Mid-tier (51-100)', 'Lower (100+)']
        surface_order = ['Hard', 'Clay', 'Grass']
        
        matrix_stats = df.groupby(['surface', 'opponent_tier']).agg(
            wins=('won', 'sum'),
            matches=('won', 'count')
        ).reset_index()
        matrix_stats['win_pct'] = matrix_stats['wins'] / matrix_stats['matches'] * 100
        
        pivot_pct = matrix_stats.pivot(index='surface', columns='opponent_tier', values='win_pct')
        pivot_pct = pivot_pct.reindex(index=surface_order, columns=tier_order)
        
        buttons.append(dict(
            label=player,
            method='update',
            args=[{'z': [pivot_pct.values]},
                  {'title': f'{tour_name} Surface x Opponent Tier: {player}'}]
        ))
    
    if fig:
        fig.update_layout(
            updatemenus=[dict(
                active=0,
                buttons=buttons,
                x=0.0,
                xanchor='left',
                y=1.02,
                yanchor='bottom',
                bgcolor='rgba(255,255,255,0.9)',
                bordercolor='#ccc',
                borderwidth=1,
                font=dict(size=11),
                pad=dict(r=10, t=10)
            )],
            margin=dict(t=80)
        )
    
    return fig

# Show for top players
fig_surf_atp = create_surface_tier_comparison(atp_records, 'ATP')
if fig_surf_atp:
    fig_surf_atp.show()

In [13]:
fig_surf_wta = create_surface_tier_comparison(wta_records, 'WTA')
if fig_surf_wta:
    fig_surf_wta.show()

## View 6: Tournament-Level Breakdown

Do players perform differently at major tournaments? This view compares performance against each opponent tier
at different tournament levels: Grand Slams, Masters 1000, and smaller events.

In [14]:
def get_tourney_category(level):
    """Categorize tournament level"""
    if level == 'G':
        return 'Grand Slam'
    elif level == 'M':
        return 'Masters 1000'
    elif level in ['A', 'T1', 'T2']:
        return 'ATP 500/250 (WTA 500+)'
    else:
        return 'Other'

def create_tournament_tier_breakdown(records_df, title, min_matches=3):
    """Create tournament level x opponent tier breakdown"""
    df = records_df.copy()
    df['tourney_category'] = df['tourney_level'].apply(get_tourney_category)
    df = df[(df['opponent_tier'] != 'Unknown') & (df['tourney_category'] != 'Other')]
    
    tier_order = ['Elite (1-10)', 'Strong (11-25)', 'Solid (26-50)', 'Mid-tier (51-100)', 'Lower (100+)']
    tourney_order = ['Grand Slam', 'Masters 1000', 'ATP 500/250 (WTA 500+)']
    
    # Calculate overall stats by tournament category and tier
    overall_stats = df.groupby(['tourney_category', 'opponent_tier']).agg(
        wins=('won', 'sum'),
        matches=('won', 'count')
    ).reset_index()
    overall_stats['win_pct'] = overall_stats['wins'] / overall_stats['matches'] * 100
    
    # Pivot
    pivot = overall_stats.pivot(index='tourney_category', columns='opponent_tier', values='win_pct')
    pivot_matches = overall_stats.pivot(index='tourney_category', columns='opponent_tier', values='matches')
    
    pivot = pivot.reindex(index=tourney_order, columns=tier_order)
    pivot_matches = pivot_matches.reindex(index=tourney_order, columns=tier_order)
    
    # Create annotations
    annotations = []
    for i, tourney in enumerate(pivot.index):
        for j, tier in enumerate(pivot.columns):
            pct = pivot.loc[tourney, tier]
            matches = pivot_matches.loc[tourney, tier]
            if pd.notna(pct) and pd.notna(matches) and matches >= min_matches:
                text = f"{pct:.0f}%<br>({int(matches)})"
                annotations.append(
                    dict(x=tier, y=tourney, text=text, showarrow=False,
                         font=dict(size=10, color='white' if pct > 55 else 'black'))
                )
    
    fig = go.Figure(data=go.Heatmap(
        z=pivot.values,
        x=pivot.columns.tolist(),
        y=pivot.index.tolist(),
        colorscale='RdYlGn',
        zmin=30,
        zmax=90,
        colorbar=dict(title='Win %')
    ))
    
    fig.update_layout(
        title=f"{title} - Overall Performance",
        xaxis_title='Opponent Tier',
        yaxis_title='Tournament Level',
        height=400,
        annotations=annotations
    )
    
    return fig

fig_tourney_atp = create_tournament_tier_breakdown(atp_records, 'ATP: Tournament Level Breakdown')
fig_tourney_atp.show()

In [15]:
fig_tourney_wta = create_tournament_tier_breakdown(wta_records, 'WTA: Tournament Level Breakdown')
fig_tourney_wta.show()

## View 7: Temporal Trends (Performance vs Elite Over Time)

How has each player's performance against top 10 opponents evolved over time?
Rising lines indicate improvement; falling lines suggest decline.

In [16]:
def create_temporal_trends(records_df, title, min_matches_per_period=3):
    """Create temporal trends of performance vs elite opponents"""
    # Filter to elite opponents only
    df = records_df[records_df['opponent_tier'] == 'Elite (1-10)'].copy()
    df['year'] = df['date'].dt.year
    
    # Calculate yearly win rates
    yearly_stats = df.groupby(['player_name', 'year']).agg(
        wins=('won', 'sum'),
        matches=('won', 'count')
    ).reset_index()
    yearly_stats['win_pct'] = yearly_stats['wins'] / yearly_stats['matches'] * 100
    
    # Filter periods with minimum matches
    yearly_stats = yearly_stats[yearly_stats['matches'] >= min_matches_per_period]
    
    # Only include players with multiple years of data
    player_years = yearly_stats.groupby('player_name')['year'].nunique()
    multi_year_players = player_years[player_years >= 2].index
    yearly_stats = yearly_stats[yearly_stats['player_name'].isin(multi_year_players)]
    
    if len(yearly_stats) == 0:
        return None
    
    fig = px.line(
        yearly_stats,
        x='year',
        y='win_pct',
        color='player_name',
        markers=True,
        hover_data=['matches'],
        labels={
            'year': 'Year',
            'win_pct': 'Win % vs Top 10',
            'player_name': 'Player',
            'matches': 'Matches'
        },
        title=title
    )
    
    fig.update_layout(
        height=500,
        legend=dict(
            orientation='v',
            x=1.02,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='#ccc',
            borderwidth=1,
            font=dict(size=10)
        ),
        margin=dict(r=150),
        yaxis=dict(range=[0, 100])
    )
    
    return fig

fig_temporal_atp = create_temporal_trends(atp_records, 'ATP: Win Rate vs Top 10 Over Time')
if fig_temporal_atp:
    fig_temporal_atp.show()

In [17]:
fig_temporal_wta = create_temporal_trends(wta_records, 'WTA: Win Rate vs Top 10 Over Time')
if fig_temporal_wta:
    fig_temporal_wta.show()

## View 8: Head-to-Head Deep Dive

Detailed records against specific top opponents, not just tiers.
Filter to show rivalries with 3+ matches and see surface breakdown within each H2H.

In [18]:
def create_h2h_matrix(matches_df, player_ids, player_names, title, min_matches=3):
    """Create head-to-head matrix between top 25 players"""
    # Build H2H records
    h2h_records = {}
    
    for _, row in matches_df.iterrows():
        winner_id = row['winner_id']
        loser_id = row['loser_id']
        
        # Only count matches between top 25 players
        if winner_id in player_ids and loser_id in player_ids:
            winner_name = player_names.get(winner_id, row['winner_name'])
            loser_name = player_names.get(loser_id, row['loser_name'])
            
            key = tuple(sorted([winner_name, loser_name]))
            if key not in h2h_records:
                h2h_records[key] = {'player1': key[0], 'player2': key[1],
                                    'p1_wins': 0, 'p2_wins': 0,
                                    'hard_p1': 0, 'hard_p2': 0,
                                    'clay_p1': 0, 'clay_p2': 0,
                                    'grass_p1': 0, 'grass_p2': 0}
            
            surface = row['surface'].lower() if pd.notna(row['surface']) else 'other'
            
            if winner_name == key[0]:
                h2h_records[key]['p1_wins'] += 1
                if surface == 'hard':
                    h2h_records[key]['hard_p1'] += 1
                elif surface == 'clay':
                    h2h_records[key]['clay_p1'] += 1
                elif surface == 'grass':
                    h2h_records[key]['grass_p1'] += 1
            else:
                h2h_records[key]['p2_wins'] += 1
                if surface == 'hard':
                    h2h_records[key]['hard_p2'] += 1
                elif surface == 'clay':
                    h2h_records[key]['clay_p2'] += 1
                elif surface == 'grass':
                    h2h_records[key]['grass_p2'] += 1
    
    h2h_df = pd.DataFrame(h2h_records.values())
    h2h_df['total_matches'] = h2h_df['p1_wins'] + h2h_df['p2_wins']
    h2h_df = h2h_df[h2h_df['total_matches'] >= min_matches]
    h2h_df = h2h_df.sort_values('total_matches', ascending=False)
    
    if len(h2h_df) == 0:
        return None
    
    # Create display strings
    h2h_df['h2h'] = h2h_df.apply(lambda x: f"{x['p1_wins']}-{x['p2_wins']}", axis=1)
    h2h_df['hard'] = h2h_df.apply(lambda x: f"{x['hard_p1']}-{x['hard_p2']}", axis=1)
    h2h_df['clay'] = h2h_df.apply(lambda x: f"{x['clay_p1']}-{x['clay_p2']}", axis=1)
    h2h_df['grass'] = h2h_df.apply(lambda x: f"{x['grass_p1']}-{x['grass_p2']}", axis=1)
    h2h_df['matchup'] = h2h_df['player1'] + ' vs ' + h2h_df['player2']
    
    # Create table
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['Matchup', 'H2H', 'Hard', 'Clay', 'Grass', 'Total'],
            fill_color='#1f77b4',
            font=dict(color='white', size=12),
            align='left'
        ),
        cells=dict(
            values=[
                h2h_df['matchup'],
                h2h_df['h2h'],
                h2h_df['hard'],
                h2h_df['clay'],
                h2h_df['grass'],
                h2h_df['total_matches']
            ],
            fill_color='white',
            align='left',
            height=30
        )
    )])
    
    fig.update_layout(
        title=f"{title} (Minimum {min_matches} matches)",
        height=max(400, len(h2h_df) * 35 + 100)
    )
    
    return fig

fig_h2h_atp = create_h2h_matrix(atp_matches, atp_player_ids, atp_player_names, 
                                'ATP: Head-to-Head Records Among Top 25')
if fig_h2h_atp:
    fig_h2h_atp.show()

In [19]:
fig_h2h_wta = create_h2h_matrix(wta_matches, wta_player_ids, wta_player_names,
                                'WTA: Head-to-Head Records Among Top 25')
if fig_h2h_wta:
    fig_h2h_wta.show()

## Summary Statistics

Key metrics for each player summarizing their opponent quality performance.

In [20]:
def create_summary_table(records_df, title):
    """Create summary statistics table"""
    df = records_df.copy()
    
    # Overall stats
    overall = df.groupby('player_name').agg(
        total_matches=('won', 'count'),
        total_wins=('won', 'sum'),
        avg_opp_rank=('opponent_rank', 'mean')
    ).reset_index()
    overall['overall_win_pct'] = (overall['total_wins'] / overall['total_matches'] * 100).round(1)
    overall['avg_opp_rank'] = overall['avg_opp_rank'].round(0)
    
    # Elite performance
    elite = df[df['opponent_tier'] == 'Elite (1-10)'].groupby('player_name').agg(
        elite_matches=('won', 'count'),
        elite_wins=('won', 'sum')
    ).reset_index()
    elite['elite_win_pct'] = (elite['elite_wins'] / elite['elite_matches'] * 100).round(1)
    
    # Merge
    summary = overall.merge(elite, on='player_name', how='left')
    summary['elite_win_pct'] = summary['elite_win_pct'].fillna(0)
    summary['elite_matches'] = summary['elite_matches'].fillna(0).astype(int)
    
    # Calculate drop-off (lower tier win% - elite win%)
    lower = df[df['opponent_tier'] == 'Lower (100+)'].groupby('player_name').agg(
        lower_wins=('won', 'sum'),
        lower_matches=('won', 'count')
    ).reset_index()
    lower['lower_win_pct'] = lower['lower_wins'] / lower['lower_matches'] * 100
    
    summary = summary.merge(lower[['player_name', 'lower_win_pct']], on='player_name', how='left')
    summary['dropoff'] = (summary['lower_win_pct'] - summary['elite_win_pct']).round(1)
    
    summary = summary.sort_values('elite_win_pct', ascending=False)
    
    # Create table
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['Player', 'Matches', 'Win %', 'Avg Opp Rank', 'vs Elite', 'Elite Win %', 'Drop-off'],
            fill_color='#1f77b4',
            font=dict(color='white', size=11),
            align='left'
        ),
        cells=dict(
            values=[
                summary['player_name'],
                summary['total_matches'],
                summary['overall_win_pct'].astype(str) + '%',
                summary['avg_opp_rank'].astype(int),
                summary['elite_matches'].astype(str) + ' matches',
                summary['elite_win_pct'].astype(str) + '%',
                summary['dropoff'].apply(lambda x: f"+{x:.0f}%" if pd.notna(x) else 'N/A')
            ],
            fill_color=[['white', '#f0f0f0'] * (len(summary) // 2 + 1)][:len(summary)],
            align='left',
            height=28
        )
    )])
    
    fig.update_layout(
        title=title,
        height=max(400, len(summary) * 32 + 100)
    )
    
    return fig

fig_summary_atp = create_summary_table(atp_records, 'ATP: Player Summary Statistics')
fig_summary_atp.show()

In [21]:
fig_summary_wta = create_summary_table(wta_records, 'WTA: Player Summary Statistics')
fig_summary_wta.show()

## Key Insights

This analysis reveals several patterns in how players perform against different quality opponents:

1. **Elite Beaters**: Players with relatively flat drop-off lines maintain strong performance regardless of opponent quality
2. **Stat Padders**: Large drop-offs between lower-tier and elite performance may indicate inflated records against weaker opponents
3. **Surface Specialists**: Some players show dramatically different performance by surface when facing top opponents
4. **Big Stage Performers**: Tournament-level breakdown reveals who rises to the occasion at Grand Slams vs smaller events
5. **Trajectory**: Temporal trends show which players are improving or declining against elite competition